In [1]:
import warnings
warnings.filterwarnings("ignore")

from google.protobuf import text_format
from brainlit.viz.visualize import napari_viewer

from ffn.utils import bounding_box_pb2
from ffn.inference import storage
from ffn.inference import inference
from ffn.inference import inference_pb2
from ffn.inference import inference_flags

import numpy as np
import h5py
import os
from skimage import io
from pathlib import Path

In [5]:
warnings.filterwarnings("ignore")
config = '''image {
  hdf5: "brainlit/images/image1.h5:raw"
}
image_mean: 64
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "brainlit/models/test_1/model.ckpt-10000"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
segmentation_output_dir: "brainlit/results/test_1/tutorial"
inference_options {
  init_activation: 0.95
  pad_value: 0.05
  move_threshold: 0.9
  min_boundary_dist { x: 1 y: 1 z: 1}
  segment_threshold: 0.6
  min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (150, 150, 150))

bbox = bounding_box_pb2.BoundingBox()
bounding_box = 'start { x:0 y:0 z:0 } size { x:150 y:150 z:150 }'
text_format.Parse(bounding_box, bbox)

runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


In [24]:
warnings.filterwarnings("ignore")
filename = "brainlit/images/image1.h5"
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())

    # Get the data
    data = list(f["raw"])
    img = np.array(data)
    # data = list(f["mask"])
    # mask = np.array(data)
    
seg, _ = storage.load_segmentation('brainlit/results/test_1/tutorial', (0, 0, 0))
seg2 = seg
seg2 = np.int64(seg2>0)
sum(sum(sum(seg2)))
napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['raw']>


In [6]:
warnings.filterwarnings("ignore")
im_num = 1
config = '''image {
    hdf5: "brainlit/images/validation/validation_%d.h5:raw"
}
image_mean: 64
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "brainlit/models/test_1/model.ckpt-10000"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
segmentation_output_dir: "brainlit/results/test_1/validation_%d"
inference_options {
    init_activation: 0.95
    pad_value: 0.05
    move_threshold: 0.9
    min_boundary_dist { x: 1 y: 1 z: 1}
    segment_threshold: 0.6
    min_segment_size: 1000
}''' % (im_num, im_num)
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (330, 330, 100))

bbox = bounding_box_pb2.BoundingBox()
bounding_box = 'start { x:0 y:0 z:0 } size { x:330 y:330 z:100 }'
text_format.Parse(bounding_box, bbox)
    
runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


In [25]:
warnings.filterwarnings("ignore")
filename = "brainlit/images/validation/validation_%d.h5" % im_num
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())

    # Get the data
    data = list(f["raw"])
    img = np.array(data)
    img = np.swapaxes(img, 0, 2)
    data = list(f["mask"])
    mask = np.array(data)

seg, _ = storage.load_segmentation('brainlit/results/test_1/validation_%d' % im_num, (0, 0, 0))
seg2 = seg
seg2 = np.int64(seg2 > 0)
sum(sum(sum(seg2)))
napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['mask', 'raw']>


In [18]:
warnings.filterwarnings("ignore")
im_num = 1
config = '''image {
    hdf5: "brainlit/images/test/test_1.h5:raw"
}
image_mean: 64
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "brainlit/models/test_1/model.ckpt-10000"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
segmentation_output_dir: "brainlit/results/test_1"
inference_options {
    init_activation: 0.95
    pad_value: 0.05
    move_threshold: 0.9
    min_boundary_dist { x: 1 y: 1 z: 1}
    segment_threshold: 0.6
    min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (330, 330, 100))

bbox = bounding_box_pb2.BoundingBox()
bounding_box = 'start { x:0 y:0 z:0 } size { x:330 y:330 z:100 }'
text_format.Parse(bounding_box, bbox)
    
runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/test_1/model.ckpt-10000


In [26]:
warnings.filterwarnings("ignore")
filename = "brainlit/images/test/test_1.h5"
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())

    # Get the data
    data = list(f["raw"])
    img = np.array(data)
    img = np.swapaxes(img, 0, 2)
    data = list(f["mask"])
    mask = np.array(data)

seg, _ = storage.load_segmentation('brainlit/results/test_1', (0, 0, 0))
seg2 = seg
seg2 = np.int64(seg2 > 0)
sum(sum(sum(seg2)))
napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['mask', 'raw']>


In [6]:
config = '''image {
    hdf5: "brainlit/images/image1.h5:raw"
}
image_mean: 64
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "brainlit/models/10_images/model.ckpt-10000"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
segmentation_output_dir: "brainlit/results/10_images"
inference_options {
    init_activation: 0.95
    pad_value: 0.05
    move_threshold: 0.9
    min_boundary_dist { x: 1 y: 1 z: 1}
    segment_threshold: 0.6
    min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (150, 150, 150))

bbox = bounding_box_pb2.BoundingBox()
bounding_box = 'start { x:0 y:0 z:0 } size { x:150 y:150 z:150 }'
text_format.Parse(bounding_box, bbox)
    
runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

C:\Users\Tanch\ffn\ffn\inference\storage.py:62: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  volume = h5py.File(path[0])[path[1]]


INFO:tensorflow:Restoring parameters from brainlit/models/10_images/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/10_images/model.ckpt-10000


In [7]:
filename = "brainlit/images/image1.h5"
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())

    # Get the data
    data = list(f["raw"])
    img = np.array(data)
    # data = list(f["mask"])
    # mask = np.array(data)
    
seg, _ = storage.load_segmentation('brainlit/results/10_images', (0, 0, 0))
seg2 = seg
seg2 = np.int64(seg2 > 0)
napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['raw']>


In [22]:
im_num = 2
config = '''image {
    hdf5: "brainlit/images/validation/validation_%d.h5:raw"
}
image_mean: 64
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "brainlit/models/10_images/model.ckpt-10000"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
segmentation_output_dir: "brainlit/results/10_images/validation_%d"
inference_options {
    init_activation: 0.95
    pad_value: 0.05
    move_threshold: 0.9
    min_boundary_dist { x: 1 y: 1 z: 1}
    segment_threshold: 0.6
    min_segment_size: 1000
}''' % (im_num, im_num)
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (330, 330, 100))

bbox = bounding_box_pb2.BoundingBox()
bounding_box = 'start { x:0 y:0 z:0 } size { x:330 y:330 z:100 }'
text_format.Parse(bounding_box, bbox)
    
runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

C:\Users\Tanch\ffn\ffn\inference\storage.py:62: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  volume = h5py.File(path[0])[path[1]]


INFO:tensorflow:Restoring parameters from brainlit/models/10_images/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/10_images/model.ckpt-10000
C:\Users\Tanch\ffn\ffn\inference\inference.py:589: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if np.any(self.segmentation[sel] > 0):
C:\Users\Tanch\ffn\ffn\inference\inference.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.seed[[slice(s, e) for s, e in zip(start, end)]])
C:\Users\Tanch\ffn\ffn\inference\inference.py:382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `

In [23]:
filename = "brainlit/images/validation/validation_%d.h5" % im_num
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())

    # Get the data
    data = list(f["raw"])
    img = np.array(data)
    img = np.swapaxes(img, 0, 2)
    data = list(f["mask"])
    mask = np.array(data)
    
seg, _ = storage.load_segmentation('brainlit/results/10_images/validation_%d' % im_num, (0, 0, 0))
seg2 = seg
seg2 = np.int64(seg2 > 0)
napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['mask', 'raw']>


In [24]:
sum(sum(sum(seg)))

0.0

In [12]:
config = '''image {
  hdf5: "brainlit/images/image1.h5:raw"
}
image_mean: 64
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "brainlit/models/5_images/model.ckpt-10000"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
segmentation_output_dir: "brainlit/results/5_images/"
inference_options {
  init_activation: 0.95
  pad_value: 0.05
  move_threshold: 0.9
  min_boundary_dist { x: 1 y: 1 z: 1}
  segment_threshold: 0.6
  min_segment_size: 1000
}'''
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (150, 150, 150))

bbox = bounding_box_pb2.BoundingBox()
bounding_box = 'start { x:0 y:0 z:0 } size { x:150 y:150 z:150 }'
text_format.Parse(bounding_box, bbox)

runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

C:\Users\Tanch\ffn\ffn\inference\storage.py:62: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  volume = h5py.File(path[0])[path[1]]


INFO:tensorflow:Restoring parameters from brainlit/models/5_images/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/5_images/model.ckpt-10000
C:\Users\Tanch\ffn\ffn\inference\inference.py:589: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if np.any(self.segmentation[sel] > 0):
C:\Users\Tanch\ffn\ffn\inference\inference.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.seed[[slice(s, e) for s, e in zip(start, end)]])
C:\Users\Tanch\ffn\ffn\inference\inference.py:382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `a

In [20]:
filename = "brainlit/images/image1.h5"
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())

    # Get the data
    data = list(f["raw"])
    img = np.array(data)
    # data = list(f["mask"])
    # mask = np.array(data)
    
seg, _ = storage.load_segmentation('brainlit/results/5_images', (0, 0, 0))
seg2 = seg
seg2 = np.int64(seg2 > 0)
napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['raw']>


In [21]:
sum(sum(sum(seg)))

2417779.0

In [14]:
im_num = 1
config = '''image {
    hdf5: "brainlit/images/validation/validation_%d.h5:raw"
}
image_mean: 64
image_stddev: 33
checkpoint_interval: 1800
seed_policy: "PolicyPeaks"
model_checkpoint_path: "brainlit/models/5_images/model.ckpt-10000"
model_name: "convstack_3d.ConvStack3DFFNModel"
model_args: "{\\"depth\\": 6, \\"fov_size\\": [11, 11, 11], \\"deltas\\": [4, 4, 4]}"
segmentation_output_dir: "brainlit/results/5_images/validation_%d"
inference_options {
    init_activation: 0.95
    pad_value: 0.05
    move_threshold: 0.9
    min_boundary_dist { x: 1 y: 1 z: 1}
    segment_threshold: 0.6
    min_segment_size: 1000
}''' % (im_num, im_num)
req = inference_pb2.InferenceRequest()
_ = text_format.Parse(config, req)

runner = inference.Runner()
runner.start(req)
canvas, alignment = runner.make_canvas((0, 0, 0), (330, 330, 100))

bbox = bounding_box_pb2.BoundingBox()
bounding_box = 'start { x:0 y:0 z:0 } size { x:330 y:330 z:100 }'
text_format.Parse(bounding_box, bbox)
    
runner.run((bbox.start.z, bbox.start.y, bbox.start.x), (bbox.size.z, bbox.size.y, bbox.size.x))

C:\Users\Tanch\ffn\ffn\inference\storage.py:62: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  volume = h5py.File(path[0])[path[1]]


INFO:tensorflow:Restoring parameters from brainlit/models/5_images/model.ckpt-10000


INFO:tensorflow:Restoring parameters from brainlit/models/5_images/model.ckpt-10000
C:\Users\Tanch\ffn\ffn\inference\inference.py:589: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if np.any(self.segmentation[sel] > 0):
C:\Users\Tanch\ffn\ffn\inference\inference.py:421: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.seed[[slice(s, e) for s, e in zip(start, end)]])
C:\Users\Tanch\ffn\ffn\inference\inference.py:382: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `a

In [18]:
filename = "brainlit/images/validation/validation_%d.h5" % im_num
with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())

    # Get the data
    data = list(f["raw"])
    img = np.array(data)
    img = np.swapaxes(img, 0, 2)
    data = list(f["mask"])
    mask = np.array(data)
    
seg, _ = storage.load_segmentation('brainlit/results/5_images/validation_%d' % im_num, (0, 0, 0))
seg2 = seg
seg2 = np.int64(seg2 > 0)
napari_viewer(img, labels=seg2)

Keys: <KeysViewHDF5 ['mask', 'raw']>


In [19]:
sum(sum(sum(seg)))

0.0